In [1]:
!pip install super_image

In [2]:
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import DataLoader 
import torchvision.transforms.functional as F
from torch import nn, optim
import time
from datasets import load_dataset

In [3]:
from super_image.models.edsr.configuration_edsr import EdsrConfig

In [8]:
from utils import adjust_learning_rate
from dataset import EvalDataset, TrainDataset
from model import edsr, ResBlock
from train import train
from evaluate import evaluate
from predict import detect_from_path, detect_from_url

## Load data

In [8]:
t_set = load_dataset('eugenesiow/Div2k', 'bicubic_x2', split='train')
e_set = load_dataset('eugenesiow/Div2k', 'bicubic_x2', split='validation')

Found cached dataset set5 (C:/Users/Admin/.cache/huggingface/datasets/eugenesiow___set5/bicubic_x2/1.0.0/e0c6fb24ad5c097bcc602e43704e322b619ce96a3bb212e085cabdafae69727f)
Found cached dataset set5 (C:/Users/Admin/.cache/huggingface/datasets/eugenesiow___set5/bicubic_x2/1.0.0/e0c6fb24ad5c097bcc602e43704e322b619ce96a3bb212e085cabdafae69727f)


In [9]:
trainset = TrainDataset(t_set)
validset = EvalDataset(e_set)

trainloader = DataLoader(trainset, batch_size=8, shuffle=True)
validloader = DataLoader(validset, batch_size=1, shuffle=True)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cpu')

## Training

In [10]:
criterion = nn.L1Loss()
scale = 2
epochs = 25
train_loss = 0
batch_num = 0
decay_lr_at = 11, 15   # decay learning rate after these many iterations
decay_lr_to = 0.1

In [11]:
from super_image.trainer_utils import EvalPrediction
from super_image.utils.metrics import compute_metrics
import gc

In [7]:
checkpoint = None
best_metric = 0
best_epoch = 0

In [13]:
def main():
    """
    Training.
    """
    global start_epoch, label_map, epoch, checkpoint, decay_lr_at, optimizer, criterion, scale

    # Initialize model or load checkpoint
    if checkpoint is None:
        config = EdsrConfig(
        scale=scale,                               
        n_resblocks=32,
        n_feats=256
    )
        start_epoch = 0
        model = edsr(config)
        optimizer = optim.Adam(model.parameters(), lr=0.0001)
        
    else:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']

    # Move to default device
    model = model.to(device)
    criterion = criterion

    print("Number of epochs: ", epochs)
    
    # Epochs
    for epoch in range(start_epoch, epochs):

        # Decay learning rate at particular epochs
        if epoch in decay_lr_at:
            adjust_learning_rate(optimizer, decay_lr_to)

        # One epoch's training
        train(train_loader=trainloader,
              valid_loader = validloader,
              model=model,
              criterion=criterion,
              optimizer=optimizer,
              scale = scale,
              epoch=epoch)

In [14]:
main()

Number of epochs:  25
Epoch: [0][0/1]	Training Time 211.303 	Loss 0.5344 (0.5344)	


KeyboardInterrupt: 

In [10]:
checkpoint = torch.load(checkpoint, map_location=torch.device('cpu'))
start_epoch = checkpoint['epoch'] + 1
print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
model = checkpoint['model']
optimizer = checkpoint['optimizer']


Loaded checkpoint from epoch 25.



In [ ]:
evaluate(model, validloader,criterion, 4)

In [ ]:
detect_from_path("test.jpg", model)

In [28]:
torch.cuda.empty_cache()
gc.collect()

0

In [29]:
import os
os.chdir(r'/kaggle/working')

!tar -czf checkpoint.pth.tar 

from IPython.display import FileLink

FileLink(r'checkpoint.pth.tar')

tar: Cowardly refusing to create an empty archive
Try 'tar --help' or 'tar --usage' for more information.


/kaggle/working/checkpoint.pth.tar